In [1]:
import py3Dmol
from rdkit import Chem
from rdkit.Chem import AllChem
from ipywidgets import interact, interactive, fixed
import ipywidgets as widgets
from IPython.display import display

In [2]:
# obj ms will have no pre calculated propery
data = [('m1','COc1ccc2[nH]c([S@@+]([O-])Cc3ncc(C)c(OC)c3C)nc2c1'),
       ('m2','COc1ccc2[nH]c([S@+]([O-])Cc3ncc(C)c(OC)c3C)nc2c1'),
       ('m3','COc1ccc2[nH]c([S+]([O-])Cc3ncc(C)c(OC)c3C)nc2c1'),
       ('m4','CCOc1ccc2[nH]c([S@@+]([O-])Cc3ncc(C)c(OCCC)c3C)nc2c1')]
ms = [(x,Chem.AddHs(Chem.MolFromSmiles(y))) for x,y in data]
params = AllChem.ETKDG()
params.numThreads=3
for nm,m in ms:
    AllChem.EmbedMultipleConfs(m,numConfs=10,params=params)
    # align to one of the ring systems:
    AllChem.AlignMolConformers(m,m.GetSubstructMatch(Chem.MolFromSmarts('c1[nH]c2ccccc2n1')))


In [3]:
def processSuppliedMolFor3DViewer(ms):
    
    try:
        moldict = dict(ms)
    except TypeError:
        
        if type(ms) is tuple:
            moldict=list()
            moldict.append(ms)
            moldict = dict(moldict)
        elif hasattr(ms, '__iter__') is False:
            moldict=list()
            moldict.append(('m0', ms))
            moldict = dict(moldict)
        elif type(ms) is list:
            ms_name=['m'+str(x) for x in range(len(ms))]
            ms2=[(ms_name[i],ms[i]) for i in range(len(ms))]
            moldict = dict(ms2)
    return moldict
    
def addMolToViewForScrolling(view, molTuple, molTupleId, confId):
    
    mol = molTuple[molTupleId]
    
    view.removeAllModels()
    
    mb = Chem.MolToMolBlock(mol,confId=confId)
    view.addModel(mb,'sdf')
    
    view.setStyle({},{'stick':{}})
    view.zoomTo()
    
    return view.update()

def browseMolConformers(mol_obj_list,confId=None):
    
    size = (400, 400)
    bgcolor = '0xeeeeee'
    view = py3Dmol.view(width=size[0],height=size[1])
    view.setBackgroundColor(bgcolor)
    view.zoomTo()
    display(view.show())
    
    # processing supplied object that contains molecules
    moldict = processSuppliedMolFor3DViewer(mol_obj_list)
    # Creating molecule listing widget # default value molecule id = 0 or m0
    molTupleId=list(moldict.keys())
    molTupleId_dropdown = widgets.Dropdown(options=molTupleId,value=molTupleId[0])
    # Creating conformer listing widget # default value confId=0
    confId_slider = widgets.IntSlider(min=0,max=9,step=1,value=0)
    # Now start interacting
    return interact(addMolToViewForScrolling,
                    view=fixed(view),
                    molTuple=fixed(moldict),
                    molTupleId=molTupleId_dropdown,
                    confId=confId_slider)

In [4]:
browseMolConformers(ms,confId=(0, 9))

<function __main__.addMolToViewForScrolling>